In [1]:

##By Payam Nasernejad

##Inputs are chicaco_unput_1 which is exctracted data from household travel survey and Chicago_att
## some inputs are defined as python lists

##taz_production and taz_attraction are the outputs of this part which will be saved as csv file in the same directory


#import pandas as pd
import pandas as pd
import numpy as np

##put inputes to the directory before running the code
path = 'F:/CIVIL 583/Project/Codes/1step/chicago_input_1.csv'
attr_path = 'F:/CIVIL 583/Project/Codes/1step/Chicago_att.csv' 

max_veh_no = 3
min_veh_no = 0
max_person_no = 4
min_person_no = 1
#max_zone_no = 10 ###you should
#min_zone_no = 1 ##from file


def generative_func(path,attr_path,max_veh_no, min_veh_no ,max_person_no, min_person_no):
    import pandas as pd
    import numpy as np
    
    ##outputs of the model; I know it's not professional to get outputs like this, but I prefer it.
    global len_veh
    global len_person
    global len_zone
    global taz_data
    global cca
    global mca
    global persons
    global vehicles   
    global hbs_trips
    
    df = pd.read_csv(attr_path)
    max_zone_no = df["Area"].max()
    min_zone_no = df["Area"].min()
    
    
    df = pd.read_csv(path)#remember to change directory everytime
    #reading HTS with persons vehicles and trips
    persons = df["persons"]
    vehicles = df["vehicles"]
    hbs_trips = df["trips"]
    zones = df["ZONE_ID"]


    
    len_veh = max_veh_no - min_veh_no +1
    len_person = max_person_no - min_person_no +1 
    len_zone = max_zone_no - min_zone_no + 1

    taz_data = np.zeros((len_person,len_veh,len_zone))
    cca_sum_matrix = np.zeros((len_person,len_veh))
    cca_count_matrix = np.zeros((len_person,len_veh))
    



    for i in range(len(persons)):
        person_no = min_person_no
        while person_no <= max_person_no:
            if persons[i] == person_no:
                
                veh_no = min_veh_no
                while veh_no <= max_veh_no:
                    if  vehicles[i] == veh_no:
                        #####
                        cca_sum_matrix[person_no - min_person_no][veh_no - min_veh_no] += hbs_trips[i]
                        cca_count_matrix[person_no - min_person_no][veh_no - min_veh_no] += 1
                        
                        zone_no = min_zone_no
                        while zone_no <= max_zone_no:
                            if zones[i] == zone_no:
                                ##taz data
                                taz_data[person_no - min_person_no][veh_no - min_veh_no][zone_no - min_zone_no] += 1
                                
                                break
                            zone_no +=1
                        
                        break
                    veh_no += 1
    
                break           
            person_no += 1
        
        
    ##cca calculation    
    cca = cca_sum_matrix/cca_count_matrix 
        
        
    horizontal_margins= cca_sum_matrix.sum(axis = 0)/cca_count_matrix.sum(axis = 0)
    vertical_margins = cca_sum_matrix.sum(axis = 1)/cca_count_matrix.sum(axis = 1)
    grand_total = cca_sum_matrix.sum()/cca_count_matrix.sum()
    
    mca_vertical_margins = -grand_total + vertical_margins
    mca_horizontal_margins = -grand_total + horizontal_margins
    
    
    ####mca computation
    mca = np.zeros((len_person,len_veh))

    for i in range(len_person):
        for j in range(len_veh):
            if (grand_total + mca_vertical_margins[i] + mca_horizontal_margins[j]) > 0:
                mca[i][j] = grand_total + mca_vertical_margins[i] + mca_horizontal_margins[j]
            else:
                mca[i][j] = 0   
    

###linear_regression
##input could be (1,0) (0,1) (1,1) (person,vehicle)
input_variable = [1,1]
def lin_reg(input_variable):
    import statsmodels.api as sm

    ###the input variables should be selected by user
    
    if input_variable == [1,1]:
        x_variables=[persons, vehicles]
    elif input_variable == [1,0]:
        x_variables=[persons]
    elif input_variable == [0,1]:
        x_variables=[vehicles]
    
    X = pd.concat(x_variables, axis = 1)
    y_variable = [hbs_trips]
    y = pd.concat(y_variable)
    
    X2 = sm.add_constant(X)
    model = sm.OLS(y, X2)
    result = model.fit()
    #model.predict(pd.DataFrame({"const":0.5, 'persons': [2,3,4], 'vehicles': [0.5, 0.75, 1.0]}))
    print(result.summary())

    regression_matrix = np.zeros([len_person, len_veh])
    for i in range(len_person):
        for j in range(len_veh):
            
            if input_variable == [1,1]:
                in_x = [1,i + min_person_no,j + min_veh_no]
            elif input_variable == [1,0]:
                in_x = [1,i + min_person_no]
            elif input_variable == [0,1]:
                in_x = [1,j + min_veh_no]
            
            
            regression_matrix[i][j] = result.predict(in_x)
    


In [9]:
generative_func(path,attr_path,max_veh_no,min_veh_no,max_person_no,min_person_no)

In [12]:
mca


array([[ 0.        ,  1.97251592,  5.7928714 ,  6.67610981],
       [ 3.55271   ,  5.63331604,  9.45367152, 10.33690994],
       [ 6.76306086,  8.8436669 , 12.66402238, 13.5472608 ],
       [11.34439782, 13.42500386, 17.24535934, 18.12859775]])

In [13]:
cca

array([[ 2.78689567,  4.13749601,  4.3356974 ,  4.63541667],
       [ 4.84855769,  6.95599112,  7.98945736,  7.8183391 ],
       [ 6.57142857,  9.96579805, 11.24912281, 11.0741483 ],
       [ 8.87878788, 14.44615385, 15.82448037, 15.41691843]])

In [14]:
##[person, vehicle] 1: include that one
input_variable=[1,1]
lin_reg(input_variable)

                            OLS Regression Results                            
Dep. Variable:                  trips   R-squared:                       0.459
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     6099.
Date:                Mon, 06 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:43:40   Log-Likelihood:                -43343.
No. Observations:               14390   AIC:                         8.669e+04
Df Residuals:                   14387   BIC:                         8.672e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1094      0.094     -1.159      0.2

In [18]:
###production matrix; it could be cca, mca or regression matrix  
generative_matrix = mca ##it could be anythiong by this dimension 

In [24]:
##poplation info from attached excel file
taz_distr_pop = np.array([[113034.826
,318633.604
,315963.49
,142406.08
],[119272.431
,336216.774
,333399.315
,150264.48
],[61757.179
,174087.166
,172628.335
,77804.32
],[99012.375
,279105.75
,276766.875
,124740
]])

taz_distr = taz_distr_pop / taz_distr_pop.sum()

taz_hh = np.array([1956561.00,
340669.00,
25293.00,
175930.00,
39882.00,
244523.00,
110860.00,
226668.00,
35140.00,
44011.60,
56134.67
])

In [25]:
taz_distr.sum()

0.9999999999999999

In [28]:
cca

array([[ 2.78689567,  4.13749601,  4.3356974 ,  4.63541667],
       [ 4.84855769,  6.95599112,  7.98945736,  7.8183391 ],
       [ 6.57142857,  9.96579805, 11.24912281, 11.0741483 ],
       [ 8.87878788, 14.44615385, 15.82448037, 15.41691843]])

In [31]:
###production matrix; it could be cca, mca or regression matrix  
generative_matrix = cca ##it could be anything with this dimension

    
def pro_att(generative_matrix,attr_path):


    area_production = np.zeros((len_person,len_veh, len_zone))
    for i in range(len_person):
        for j in range(len_veh):
            for k in range(len_zone):
                area_production[i][j][k] = taz_distr[i][j]*taz_hh[k] * generative_matrix[i][j]
    

    taz_production = area_production.sum(axis=0).sum(axis=0)


    ####attraction matrix
    
    df = pd.read_csv(attr_path)
    rate = df["Rate"]
    employment = df["employment"]
    attraction = rate * employment
    balance_factor = taz_production.sum()/attraction.sum()
    taz_attraction = np.array(attraction * balance_factor)



    ###saving attraction and production files in the same directory of this folder
    
    from numpy import savetxt
    
    savetxt("taz_production.csv", taz_production, delimiter = ',')
    savetxt("taz_attraction.csv", taz_attraction, delimiter = ',')
    print(taz_production.sum())
    print(taz_attraction.sum())
    
    
pro_att(generative_matrix,attr_path)

27955981.471042123
27955981.47104213


In [77]:
area_production = np.zeros((len_person,len_veh, len_zone))
for i in range(len_person):
    for j in range(len_veh):
        for k in range(len_zone):
            area_production[i][j][k] = taz_distr[i][j]*taz_hh[k] * generative_matrix[i][j]

In [78]:
cca.shape

(4, 4)

In [79]:
taz_pop.shape

(4, 4)

In [80]:
area_production.shape

(4, 4, 11)

In [7]:
#this part is frontend, no codes are developed here. ## not part of the coarse project.
from tkinter import *
from PIL import Image, ImageTk
from tkinter import filedialog

root = Tk()
root.title("4 step moleding")
#root.geometry("500x500")

#root.iconbitmap("C:/Users/Payam/Desktop/ubc-logo.png")

def first_click():
    first_top = Toplevel()
    first_top.title("First Step")
    
    
    
    def file_open():
        first_top.filename = filedialog.askopenfilename(initialdir="c:", title="select a file",filetypes=(("csv files","*.csv"),("all files","*.*")) )
        path = first_top.filename
        mca_cca(path)
    
    def cca_click():
        cca_label.grid(row=2, column = 0, columnspan = 2)
    def mca_click():
        mca_label.grid(row=2, column = 2, columnspan = 2)
      
    #first_Top
    #method will be used in attraction and production
    method = IntVar()
    #method.set("1")
    ccar = Radiobutton(first_top, text="CCA", variable=method, value = 0)
    mcar = Radiobutton(first_top, text="MCA", variable=method, value = 1)

    
    
    
    open_button = Button(first_top, text = "Select File",padx = 50, command = file_open)
    lbl = Label(first_top, text = "Developed by: Payam Nasernejad")
    cca = Button(first_top, text = "CCA",padx= 40,pady = 20, command = cca_click)
    mca = Button(first_top , text = "MCA",padx= 40,pady = 20, command = mca_click)
    
    global mca_img
    mca_img = ImageTk.PhotoImage(Image.open("mca_plot.png"))
    mca_label = Label(first_top, image=mca_img)
    global cca_img
    cca_img = ImageTk.PhotoImage(Image.open("cca_plot.png"))
    cca_label = Label(first_top, image= cca_img)  
    
    ccar.grid(row = 1, column = 1)
    cca.grid(row = 1, column = 0)
    mcar.grid(row = 1, column = 3)
    mca.grid(row = 1 , column = 2)
    lbl.grid(row=0, column =0, columnspan = 4) 
    open_button.grid(row=3, column = 0, columnspan = 4)

    global est_method
    est_method = method.get()    

#function before this cell

def myclick():
    return 



##root 





lbl = Label(root, text = "Developed by: Payam Nasernejad")
first_step = Button(root, text = "First Step",padx= 40,pady = 20, command = first_click)
second_step = Button(root, text = "Second Step",padx= 40,pady = 20, command = myclick , state= "disabled")
third_step = Button(root, text = "Third Step",padx= 40,pady = 20,command = myclick , state= "disabled")
fourth_step = Button(root, text = "Fourth Step", padx= 40,pady = 20,command = myclick , state= "disabled")


lbl.grid(row=1, column =0, columnspan = 4) 
first_step.grid(row = 0, column = 0)
second_step.grid(row = 0, column = 1)
third_step.grid(row = 0, column = 2)
fourth_step.grid(row = 0, column = 3)






root.mainloop()

0